In [ ]:
import numpy as np
import os
import tensorflow as tf

from scipy.stats import gmean
from sklearn.metrics import confusion_matrix
from src.data.synthetic_gen import DataLoader
from src.models.classification import NeuralNetClassifier
from src.models.classification.evaluate_models import ClassificationAnalysis
from src.models.classification.train_models import ClassificationTraining

tf.random.set_seed(13)

In [2]:
dl = DataLoader(problem="classification", dataset_size=1)
datasets, minmax = dl.load_cross_validation_datasets()

train_data = datasets["train"]
valid_data = datasets["valid"]

### Training best neural network

- Model: Rede Neural #11
- Parameters
    - hidden units: [128, 128]
    - activation: relu
    - learning rate: 0.001
    - epochs: 500
    - batch size: 32

In [ ]:
# Train best neural network
# Rede Neural #11
learning_rate = 0.001
epochs = 500
batch_size = 32

params = {"hidden_units": [128, 128], "activation": "relu"}

models = []
for i, (train, valid) in enumerate(zip(train_data, valid_data)):

    train_features, train_labels = train["features"], train["targets"]
    valid_features, valid_labels = valid["features"], valid["targets"]

    features, labels = train_features.values, train_labels.values
    train_ds = (
        tf.data.Dataset.from_tensor_slices((features, labels))
        .shuffle(10000)
        .batch(batch_size)
    )

    features, labels = valid_features.values, valid_labels.values
    valid_ds = tf.data.Dataset.from_tensor_slices((features, labels)).batch(batch_size)

    loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.CategoricalAccuracy()
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    callbacks = [
        tf.keras.callbacks.ReduceLROnPlateau(),
        # patience 10% of epochs size
        tf.keras.callbacks.EarlyStopping(min_delta=0.0001, patience=50),
    ]
    model = NeuralNetClassifier(**params)
    model.compile(optimizer=optimizer, loss=loss_object, metrics=[accuracy])
    h = model.fit(
        train_ds,
        epochs=epochs,
        validation_data=valid_ds,
        callbacks=callbacks,
        verbose=1,  # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch
    )

    models.append(model)

    models_folder = os.path.join("notebooks", "classification_models")
    if not os.path.isdir(models_folder):
        os.mkdir(models_folder)

    model.save(os.path.join(models_folder, f"model_fold={i+1}.keras"))

Epoch 1/500
333/333 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - categorical_accuracy: 0.8286 - loss: 0.4718 - val_categorical_accuracy: 0.9406 - val_loss: 0.1371 - learning_rate: 0.0010
Epoch 2/500
333/333 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - categorical_accuracy: 0.9588 - loss: 0.1192 - val_categorical_accuracy: 0.9786 - val_loss: 0.0797 - learning_rate: 0.0010
Epoch 3/500
333/333 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - categorical_accuracy: 0.9696 - loss: 0.0835 - val_categorical_accuracy: 0.9762 - val_loss: 0.0711 - learning_rate: 0.0010
Epoch 4/500
333/333 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - categorical_accuracy: 0.9761 - loss: 0.0677 - val_categorical_accuracy: 0.9834 - val_loss: 0.0639 - learning_rate: 0.0010
Epoch 5/500
333/333 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - categorical_accuracy: 0.9763 - loss: 0.0661 - val_categorical_accuracy: 0.9857 - val_loss: 0.0511 - learning_rate: 0.0010
Epoch 6/500
333/333 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - categorical_accuracy: 0.9789 - loss: 0.0579 - val_categorica

### Evaluation

In [4]:
confusion_matrix_values = []
accuracies = []
sensitivities = []
sp_indexes = []

for valid, model in zip(valid_data, models):
    valid_features, valid_labels = valid["features"], valid["targets"]

    X_valid = tf.convert_to_tensor(valid_features)
    probs = tf.convert_to_tensor(valid_labels)
    y_valid = tf.argmax(probs, axis=1)

    logits = model(X_valid)
    probs_hat = tf.nn.softmax(logits)
    y_valid_hat = tf.argmax(probs_hat, axis=1)

    # Confusion matrix
    cm = confusion_matrix(y_valid, y_valid_hat)
    confusion_matrix_values.append(cm)

    # Accuracy
    accuracies.append(np.diag(cm).sum() / cm.sum())

    # Sensitivity
    sens = np.diag(cm) / cm.sum(axis=1)
    sensitivities.append(sens)

    # SP Index
    sp_indexes.append(np.sqrt(np.mean(sens) * gmean(sens)))

confusion_matrix_values = np.array(confusion_matrix_values)
accuracies = np.array(accuracies) * 100
sensitivities = np.array(sensitivities) * 100
sp_indexes = np.array(sp_indexes) * 100

In [6]:
# print(confusion_matrix_values)
print(f"Accuracy = {accuracies.mean():.2f} +/- {accuracies.std() / np.sqrt(10 - 1):.2f} %")

mean_sens = sensitivities.mean(axis=0)
std_err_sens = sensitivities.std(axis=0) / np.sqrt(10 - 1)
print(f"Sensitivity [V] = {mean_sens[0]:.2f} +/- {std_err_sens[0]:.2f} %")
print(f"Sensitivity [VL] = {mean_sens[1]:.2f} +/- {std_err_sens[1]:.2f} %")
print(f"Sensitivity [L] = {mean_sens[2]:.2f} +/- {std_err_sens[2]:.2f} %")

print(f"SP Index = {sp_indexes.mean():.2f} +/- {sp_indexes.std() / np.sqrt(10 - 1):.2f} %")

Accuracy = 99.34 +/- 0.10 %
Sensitivity [V] = 99.33 +/- 0.26 %
Sensitivity [VL] = 99.11 +/- 0.14 %
Sensitivity [L] = 99.59 +/- 0.18 %
SP Index = 99.34 +/- 0.10 %


In [ ]:
samples_per_composition = 30

training = ClassificationTraining(samples_per_composition=samples_per_composition)
analysis = ClassificationAnalysis(samples_per_composition=samples_per_composition)

results = training.load_training_models()
indices = analysis.load_performance_indices()

outputs = [output for i, output in enumerate(results["outputs"]) if i in models]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker


def plot_confusion_matrix(cm, ax, model_name):
    cm = cm / np.sum(cm, axis=2)[:, :, None]
    cm_mean = np.mean(cm, axis=0)
    cm_std = np.std(cm, axis=0) / np.sqrt(10 - 1)

    ms = ax.matshow(cm_mean, alpha=0.5, cmap="Greys")
    for ii in range(cm_mean.shape[0]):
        for jj in range(cm_mean.shape[1]):
            text = f"{cm_mean[ii, jj] * 100:1.2f} \\textpm \n {cm_std[ii, jj] * 100:1.2f} \%"
            ax.text(x=jj, y=ii, s=text, va="center", ha="center", size="x-large")

    cbar = ax.figure.colorbar(ms, ax=ax, shrink=0.675, format=ticker.PercentFormatter(xmax=1))
    ax.set_xlabel("Classes Estimadas")
    # ax.set_ylabel("Classes Reais")
    ax.set_title(model_name)

    # 0: gas, 1: mix, 2: oil
    ax.set_xticks([0, 1, 2], ["Vapor", "Mistura", "Líquido"])
    ax.set_yticks([0, 1, 2], ["Vapor", "Mistura", "Líquido"])
    ax.grid(False)


models = [28]
output = [output for i, output in enumerate(results["outputs"]) if i in models][0]
model_name = [res["model_name"].replace("#", "\#") for res in outputs][0]

f, axs = plt.subplots(1, len(models), figsize=(5 * len(models), 5))

cm = indices["confusion_matrix"][:, models[0], :, :].astype("int16")
plot_confusion_matrix(cm, axs[0], model_name)

f.supylabel("Classes Reais")
f.tight_layout()

# plt.savefig(os.path.join("data", "images", "comparing_confusion_matrices.png"), dpi=600)
plt.show()